In [ ]:
import pandas as pd
import pyodbc
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Kết nối tới SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-IFEAUMD;'
    'DATABASE=Anime_CauTruc;'
    'UID=...;'
    'PWD=...'
)
# Truy vấn dữ liệu
query = """
SELECT a.AnimeName,     a.Synopsis, 
       a.Episodes,      a.AnimeStatus,   
       a.Duration_min,  g.GenreName, 
       s.StudioName,    r.Rating,  
       p.Premiered,     t.AnimeType,  
       st.Ranking,      st.Popularity
FROM Anime a
JOIN Anime_genres ag ON a.AnimeID = ag.AnimeID
JOIN Genres g ON ag.GenresID = g.GenresID
JOIN Anime_Studio asd ON a.AnimeID = asd.AnimeID
JOIN Studio s ON asd.StudioID = s.StudioID
JOIN Rating r ON a.RatingID = r.RatingID
JOIN Statistic st ON a.StatisticID = st.StatisticID
JOIN Premiered p ON a.PremieredID = p.PremieredID
JOIN AnimeType t ON a.AnimeTypeID = t.AnimeTypeID
    """ 
df_f = pd.read_sql_query(query, conn)

# Đóng kết nối
conn.close()

C:\Users\OS\AppData\Local\Temp\ipykernel_13772\3082203111.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_f = pd.read_sql_query(query, conn)


In [77]:
df_f

,AnimeName,Synopsis,Episodes,AnimeStatus,Duration_min,GenreName,StudioName,Rating,Premiered,AnimeType,Ranking,Popularity
0,Sousou no Frieren,During their decade-long quest to defeat the D...,28,Finished Airing,24.0,Adventure,Madhouse,PG-13,Fall 2023,TV,1,200
1,Sousou no Frieren,During their decade-long quest to defeat the D...,28,Finished Airing,24.0,Drama,Madhouse,PG-13,Fall 2023,TV,1,200
2,Sousou no Frieren,During their decade-long quest to defeat the D...,28,Finished Airing,24.0,Fantasy,Madhouse,PG-13,Fall 2023,TV,1,200
3,Sousou no Frieren,During their decade-long quest to defeat the D...,28,Finished Airing,24.0,Shounen,Madhouse,PG-13,Fall 2023,TV,1,200
4,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,64,Finished Airing,24.0,Action,Bones,R-17+,Spring 2009,TV,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
45268,Naze Nani Dendrogram Episode 0,Episode 0 of series of short specials where Ch...,1,Finished Airing,1.0,Fantasy,N/A,PG-13,N/A,ONA,11316,15064
45269,Naze Nani Dendrogram Episode 0,Episode 0 of series of short specials where Ch...,1,Finished Airing,1.0,Video Game,N/A,PG-13,N/A,ONA,11316,15064
45270,Pankunchi,Pankun is a baby panda who was abandoned in a ...,10,Finished Airing,2.0,Adventure,Sanrio,G,N/A,ONA,11317,15986
45271,Pankunchi,Pankun is a baby panda who was abandoned in a ...,10,Finished Airing,2.0,Anthropomorphic,Sanrio,G,N/A,ONA,11317,15986


In [78]:
# Gộp các giá trị cho GenreName và StudioName
df = df_f.groupby(
    ['AnimeName', 'Synopsis', 'Episodes', 'AnimeStatus', 'Duration_min', 
     'Rating', 'Premiered', 'AnimeType', 'Ranking', 'Popularity'], as_index=False
).agg({
    'GenreName': lambda x: ', '.join(x.unique()),      # Gộp GenreName
    'StudioName': lambda x: ', '.join(x.unique())      # Gộp StudioName
})



In [79]:
df.to_csv('Anime.csv', index=False)

In [80]:
display(df)

,AnimeName,Synopsis,Episodes,AnimeStatus,Duration_min,Rating,Premiered,AnimeType,Ranking,Popularity,GenreName,StudioName
0,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",Short episode bundled with the limited edition...,1,Finished Airing,15.0,PG-13,N/A,OVA,4902,5016,"Comedy, Fantasy, School",Production I.G
1,"""Bungaku Shoujo"" Memoire",Episodes which depict the background stories o...,3,Finished Airing,24.0,PG-13,N/A,OVA,3029,4179,"Drama, Romance, School",Production I.G
2,"""Bungaku Shoujo"" Movie","The protagonist of the story, Konoha Inoue, is...",1,Finished Airing,100.0,PG-13,N/A,Movie,2623,2701,"Drama, Mystery, Romance, School",Production I.G
3,"""Eiyuu"" Kaitai","Uro Yamada works as part of the ""Hero Retireme...",1,Finished Airing,27.0,PG-13,N/A,OVA,10045,8476,"Comedy, Drama, Isekai",Zexcs
4,"""Oshi no Ko""","In the entertainment world, celebrities often ...",11,Finished Airing,30.0,PG-13,Spring 2023,TV,83,213,"Award Winning, Drama, Supernatural, Reincarnat...",Doga Kobo
...,...,...,...,...,...,...,...,...,...,...,...,...
11195,xxxHOLiC Movie: Manatsu no Yoru no Yume,"Summer break has arrived, but while his other ...",1,Finished Airing,60.0,PG-13,N/A,Movie,731,2737,"Comedy, Drama, Mystery, Supernatural, Psycholo...",Production I.G
11196,xxxHOLiC Rou,Ten years after the events of xxxHOLiC Shunmuk...,2,Finished Airing,34.0,PG-13,N/A,OVA,394,2666,"Drama, Mystery, Supernatural, Seinen",Production I.G
11197,xxxHOLiC Shunmuki,"For the appropriate price, your dearest wish c...",2,Finished Airing,26.0,PG-13,N/A,OVA,591,2638,"Comedy, Drama, Mystery, Supernatural, Seinen",Production I.G
11198,xxxHOLiC◆Kei,Kimihiro Watanuki's life was never really norm...,13,Finished Airing,24.0,PG-13,Spring 2008,TV,377,1621,"Comedy, Drama, Mystery, Supernatural, Seinen",Production I.G


In [81]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11200 entries, 0 to 11199
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AnimeName     11200 non-null  object 
 1   Synopsis      11200 non-null  object 
 2   Episodes      11200 non-null  int64  
 3   AnimeStatus   11200 non-null  object 
 4   Duration_min  11200 non-null  float64
 5   Rating        11200 non-null  object 
 6   Premiered     11200 non-null  object 
 7   AnimeType     11200 non-null  object 
 8   Ranking       11200 non-null  int64  
 9   Popularity    11200 non-null  int64  
 10  GenreName     11200 non-null  object 
 11  StudioName    11200 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 1.0+ MB
None


In [82]:
# Tiền xử lý dữ liệu
import re
df['GenreName'] = df['GenreName'].apply(lambda x: re.sub(r',', ' ', x.lower()) if isinstance(x, str) else '')
df['StudioName'] = df['StudioName'].apply(lambda x: re.sub(r',', ' ', x.lower()) if isinstance(x, str) else '')

# Tạo cột mô tả tổng hợp
df['Description'] = df['StudioName'] + ' ' + \
                    df['Premiered'] + ' ' + \
                    df['Ranking'].astype(str) + ' ' + \
                    df['GenreName'] + ' ' + \
                    df['Episodes'].astype(str) + ' ' + \
                    df['AnimeStatus'] + ' ' + \
                    df['Duration_min'].astype(str) + ' ' + \
                    df['Popularity'].astype(str) + ' ' + \
                    df['Rating'].astype(str)

# Tạo DataFrame mới chỉ gồm các cột cần thiết
new_df = df[['AnimeName', 'Description', 'Synopsis']]

In [83]:
# Sử dụng TfidfVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english', max_features=1500)
genre_vec = tfidf_vec.fit_transform(new_df['Description'])

In [84]:
# Hàm đề xuất phim dựa trên cosine similarity
def get_recommend(title, cosine_sim, length=10):
    # Tạo một series để ánh xạ tên phim với chỉ số
    movie_title_series = pd.Series(df.index, index=new_df['AnimeName']).drop_duplicates()

    # Lấy chỉ số của bộ phim dựa trên tên
    if title not in movie_title_series:
        raise ValueError(f"'{title}' không tồn tại trong dữ liệu.")
    movie_idx = movie_title_series[title]

    # Tính toán điểm tương đồng
    sim_scores = list(enumerate(cosine_sim[movie_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:length+1]  # Bỏ chính phim được chọn (phim đầu tiên)

    # Lấy tên phim và nội dung
    movie_indices = [i[0] for i in sim_scores]
    recommendations = new_df.iloc[movie_indices][['AnimeName', 'Synopsis']]
    return recommendations


In [85]:
# Đề xuất dựa trên danh sách phim người dùng đã chọn
def get_recommend_by_user_list(user_list, length=10):
    # Kiểm tra đầu vào
    if not user_list:
        raise ValueError("Danh sách phim người dùng không được để trống.")

    # Tạo vector người dùng bằng cách lấy trung bình vectơ của các phim
    user_vec = np.mean(
        [genre_vec[new_df[new_df['AnimeName'] == movie].index[0]].toarray() 
         for movie in user_list if movie in new_df['AnimeName'].values],
        axis=0
    )

    # Tính toán độ tương đồng giữa vector người dùng và các bộ phim
    cosine = cosine_similarity(user_vec, genre_vec)

    # Sắp xếp các bộ phim theo độ tương đồng giảm dần
    sim_scores = list(enumerate(cosine[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Loại bỏ các phim đã chọn trong danh sách người dùng
    sim_scores = [score for score in sim_scores if new_df.iloc[score[0]]['AnimeName'] not in user_list]
    sim_scores = sim_scores[:length]  # Chỉ lấy số lượng phim cần đề xuất

    # Lấy tên phim và nội dung
    movie_indices = [i[0] for i in sim_scores]
    recommendations = new_df.iloc[movie_indices][['AnimeName', 'Synopsis']]
    return recommendations



In [86]:
name = input('Nhap ten phim ban da coi:')
# Tính độ tương đồng cosine
cos_similar = cosine_similarity(genre_vec, genre_vec)
# từ 1 phim đề xuất ra
# Ví dụ sử dụng:
print(f"Kết quả đề xuất cho phim {name}:")
recommendation_1 = get_recommend(name, cos_similar, length=10)
display(recommendation_1)

Kết quả đề xuất cho phim One Piece:


,AnimeName,Synopsis
1969,Digimon Adventure,When a group of seven children go to summer ca...
8169,Saint Seiya: Meiou Hades Juuni Kyuu-hen - Yomi...,Is a summary of the Saga Hades Sanctuary from ...
2244,Dragon Quest: Dai no Daibouken (2020): Bouken ...,Recap of first 37 episodes of Dragon Quest: Da...
7123,One Piece: Dai Tettei Kaibou! Gekitou! 5-nin n...,Special that recaps the fight on the Skull Dom...
7124,One Piece: Dai Tettei Kaibou! Kouzuki Oden Den...,"Alongside Yamato, Momonosuke Kozuki recounts t..."
7126,One Piece: Densetsu no Log! Akagami no Shanks!,As the world anticipates the legendary diva Ut...
7138,One Piece: Kinkyuu Kikaku One Piece Kanzen Kou...,"This is the first recapitulation episode, cove..."
7121,One Piece: Dai Gekisen Tokushuu! Zoro vs. Ooka...,This is the 7th recap special of the Wano arc.
7125,One Piece: Dai Tokushuu! Momonosuke no Mei Sho...,This is the 11th recap special of the Wano arc.
7139,One Piece: Kizamareru Rekishi! Gekidou no Shin...,The third recap special of Egghead arc. Zoro a...


In [87]:
#  thử cái ni đi đưa ít phim để chạy thử
print("\nKết quả đề xuất dựa trên danh sách người dùng:")
user_selected_movies = ['Drangon Ball', 'Bleach', 'Boku no Hero Academia']
recommendations = get_recommend_by_user_list(user_selected_movies, length=10)
display(recommendations)



Kết quả đề xuất dựa trên danh sách người dùng:


,AnimeName,Synopsis
1061,Boku no Hero Academia the Movie 1: Futari no H...,Episode 1 is based on one-shot manga released ...
1065,Boku no Hero Academia the Movie 3: World Heroe...,"Adaptation of ""No.XXX Hawks: SOOTHE"" side-chap..."
1067,Boku no Hero Academia: Hero Note,Recap of Boku no Hero Academia that aired a we...
1068,Boku no Hero Academia: Ikinokore! Kesshi no Su...,The stage this time is a survival training cou...
1069,Boku no Hero Academia: Memories,Recap of Boku no Hero Academia leading up to t...
1071,Boku no Hero Academia: Training of the Dead,"Returning from their internships, the students..."
1070,Boku no Hero Academia: Sukue! Kyuujo Kunren!,UA High School must regain the public's confid...
4724,Kekkai Sensen: Ousama no Restaurant no Ousama,"For a rare occasion, the members of Libra have..."
6451,Mob Psycho 100: Dai Ikkai Rei toka Soudansho I...,The staff of the Spirits and Such Consultation...
3929,Hunter x Hunter Pilot,"Gon Freecss, a young boy living on Whale Islan..."
